In [2]:
import sys
import os
import argparse
import json

# 添加项目路径
sys.path.append('/Users/haha/Story')  # 你的项目根目录

# 导入所有必要模块
from src.constant import output_dir
from src.utils.utils import save_md, save_json, load_json, extract_plot_list, generate_response, convert_json
from src.generation.outline_generator import generate_outline
from src.generation.chapter_reorder import reorder_chapters
from src.generation.generate_characters import generate_characters_v1
from src.generation.expand_story import expand_story_v1
from src.compile_story import compile_full_story_by_sentence, compile_full_story_by_chapter
from src.enhance_story import enhance_story_with_transitions, polish_dialogues_in_story
from src.generation.dialogue_inserter import analyze_dialogue_insertions, run_dialogue_insertion, analyze_dialogue_insertions_v2
from src.utils.utils import extract_behavior_llm, convert_dialogue_dict_to_list
from src.sync.plot_sync_manager import sync_plot_and_dialogue_from_behavior
from src.sync.auto_propagate_plot_update import auto_propagate_plot_update
from src.analysis.character_state_tracker import run_character_state_tracker
from src.utils.logger import append_log, build_log_record, build_simple_log, init_log_path
from src.version_namer import build_version_name 

print("✅ 所有模块导入成功")

#

✅ 所有模块导入成功


In [2]:
# 设置项目路径
project_path = "/Users/haha/Story"
sys.path.append(project_path)

# 导入必要的函数
try:
    from src.utils.utils import generate_response, convert_json
    print("✅ 成功导入项目模块")
except ImportError as e:
    print(f"⚠️ 导入失败: {e}")
    # 模拟函数
    def generate_response(messages):
        return '{"next_speaker": "NONE"}'
    def convert_json(text):
        try:
            return json.loads(text)
        except:
            return {}

# 测试数据
test_sentence_context = "小红帽刚踏出飞船舱门,星际森林的低语与机械残骸的回声交织在耳边"
test_candidates = ["小红帽", "赛博狼"]
test_characters = [
    {"name": "小红帽", "description": "勇敢的快递员"},
    {"name": "赛博狼", "description": "危险的黑客"}
]

print("🧪 测试环境准备完成")

✅ 成功导入项目模块
🧪 测试环境准备完成


In [3]:
# Cell 2: 原版函数（从你的代码复制）
def generate_dialogue_for_insertion_original(sentence_context, candidate_characters, full_plot, character_personality):
    """原版函数，用于对比"""
    print(f"\n🔍 原版：开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""
    MAX_ROUNDS = 5  # 测试时降低轮数

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，请基于以下剧情做出第一句发言。
剧情背景是：{sentence_context}
你可以向其他角色说话：{[c for c in candidate_characters if c != speaker]}
用以下json格式返回：
{{"dialogue": "...", "action": "..."}}"""
    }]
    
    response = generate_response(prompt_first)
    parsed = convert_json(response)
    
    if isinstance(parsed, dict) and "dialogue" in parsed:
        spoken_line = parsed.get("dialogue", "")
        action = parsed.get("action", "")
        dialogue_list.append({
            "speaker": speaker,
            "dialogue": spoken_line,
            "action": action
        })
        history += f"{speaker}: {spoken_line}\n"

    # 多轮对话循环
    round_count = 0
    while round_count < MAX_ROUNDS:
        round_count += 1
        print(f"\n  🔄 原版第{round_count}轮对话")
        
        # 原版判断逻辑
        speaker_prompt = [{
            "role": "system",
            "content": f"""你是故事编剧。
当前剧情：{sentence_context}
当前已有对话历史：
{history}

只能从以下角色中选择发言：{candidate_characters}
请判断下一位发言人是谁？如果不需要继续对话，返回"NONE"。
格式：{{"next_speaker": "角色"}}"""
        }]
        
        next_res = generate_response(speaker_prompt)
        next_data = convert_json(next_res)
        next_speaker = next_data.get("next_speaker", "NONE")
        
        print(f"    📥 原版判断: {next_speaker}")
        
        if next_speaker == "NONE" or next_speaker not in candidate_characters:
            print(f"原版结束对话，next_speaker={next_speaker}")
            break

        # 生成发言内容（简化版）
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
以及对话历史：{history}
继续说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_reply)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": next_speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{next_speaker}: {spoken_line}\n"
    
    print(f"  ✅ 原版生成了{len(dialogue_list)}条对话")
    return dialogue_list

In [4]:
# Cell 3: 改进版本1 - 双重判断法
def generate_dialogue_for_insertion_v1(sentence_context, candidate_characters, full_plot, character_personality):
    """改进版本1：双重判断法"""
    print(f"\n🔍 改进版1：开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""
    MAX_ROUNDS = 10  # 稍微提高上限

    # 第一步：分析对话目标
    goal_prompt = [{
        "role": "system",
        "content": f"""你是故事编剧。分析以下剧情片段需要用对话表达什么：

剧情：{sentence_context}
可用角色：{candidate_characters}

请分析：这段剧情最需要通过对话表达什么？预期几轮对话比较合适？

格式：{{"goal": "对话目标描述", "expected_rounds": 数字}}"""
    }]
    
    goal_response = generate_response(goal_prompt)
    goal_data = convert_json(goal_response)
    dialogue_goal = goal_data.get("goal", "推进剧情")
    expected_rounds = goal_data.get("expected_rounds", 3)
    
    print(f"  🎯 对话目标: {dialogue_goal}")
    print(f"  📊 预期轮数: {expected_rounds}")

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，请基于以下剧情做出第一句发言。
剧情背景：{sentence_context}
对话目标：{dialogue_goal}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue": "...", "action": "..."}}"""
    }]
    
    response = generate_response(prompt_first)
    parsed = convert_json(response)
    
    if isinstance(parsed, dict) and "dialogue" in parsed:
        spoken_line = parsed.get("dialogue", "")
        action = parsed.get("action", "")
        dialogue_list.append({
            "speaker": speaker,
            "dialogue": spoken_line,
            "action": action
        })
        history += f"{speaker}: {spoken_line}\n"

    # 改进的多轮对话循环
    round_count = 0
    while round_count < MAX_ROUNDS:
        round_count += 1
        print(f"\n  🔄 改进版1第{round_count}轮对话")
        
        # 🎯 改进的判断逻辑：双重判断
        judge_prompt = [{
            "role": "system",
            "content": f"""你是故事编剧。请分析当前对话状态：

【剧情背景】：{sentence_context}
【对话目标】：{dialogue_goal}
【预期轮数】：{expected_rounds}
【当前轮数】：{round_count}
【对话历史】：
{history}

请回答两个问题：
1. 对话目标是否已经达成？(0-10分评分)
2. 如果未充分达成，下一个发言人是谁？

可选角色：{candidate_characters}

格式：{{"goal_achieved": 分数, "should_continue": true/false, "next_speaker": "角色名或NONE", "reason": "判断理由"}}"""
        }]
        
        judge_res = generate_response(judge_prompt)
        judge_data = convert_json(judge_res)
        
        goal_achieved = judge_data.get("goal_achieved", 5)
        should_continue = judge_data.get("should_continue", True)
        next_speaker = judge_data.get("next_speaker", "NONE")
        reason = judge_data.get("reason", "")
        
        print(f"    📊 目标达成度: {goal_achieved}/10")
        print(f"    🤔 是否继续: {should_continue}")
        print(f"    👤 下一发言人: {next_speaker}")
        print(f"    💭 理由: {reason}")
        
        # 🎯 改进的停止条件
        if (not should_continue or 
            goal_achieved >= 7 or  # 目标达成度高
            next_speaker == "NONE" or 
            next_speaker not in candidate_characters or
            round_count >= expected_rounds + 2):  # 超过预期轮数
            print(f"改进版1结束对话: goal_achieved={goal_achieved}, should_continue={should_continue}")
            break

        # 生成发言内容
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
对话目标：{dialogue_goal}
对话历史：{history}
继续说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_reply)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": next_speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{next_speaker}: {spoken_line}\n"
    
    print(f"  ✅ 改进版1生成了{len(dialogue_list)}条对话")
    return dialogue_list

In [5]:
# Cell 4: 改进版本2 - 简化判断法
def generate_dialogue_for_insertion_v2(sentence_context, candidate_characters, full_plot, character_personality):
    """改进版本2：简化但更明确的判断"""
    print(f"\n🔍 改进版2：开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，基于剧情做出第一句发言：{sentence_context}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue": "...", "action": "..."}}"""
    }]
    
    response = generate_response(prompt_first)
    parsed = convert_json(response)
    
    if isinstance(parsed, dict) and "dialogue" in parsed:
        spoken_line = parsed.get("dialogue", "")
        action = parsed.get("action", "")
        dialogue_list.append({
            "speaker": speaker,
            "dialogue": spoken_line,
            "action": action
        })
        history += f"{speaker}: {spoken_line}\n"

    # 🎯 极简判断循环
    round_count = 0
    while round_count < 6:  # 硬限制
        round_count += 1
        print(f"\n  🔄 改进版2第{round_count}轮对话")
        
        # 🎯 简化但明确的判断
        judge_prompt = [{
            "role": "system",
            "content": f"""剧情：{sentence_context}
已有对话：{history}

这段对话是否已经足够表达剧情内容了？
- 如果已经足够，返回 "STOP"
- 如果还需要继续，从以下角色选择下一个发言人：{candidate_characters}

只返回角色名或"STOP"，不要解释。"""
        }]
        
        judge_res = generate_response(judge_prompt)
        next_speaker = judge_res.strip().strip('"').strip("'")
        
        print(f"    📥 简化判断: {next_speaker}")
        
        if (next_speaker == "STOP" or 
            next_speaker not in candidate_characters or
            len(history.split('\n')) >= 6):  # 额外保护
            print(f"改进版2结束对话: {next_speaker}")
            break

        # 生成发言
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
和对话历史：{history}
说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_reply)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": next_speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{next_speaker}: {spoken_line}\n"
    
    print(f"  ✅ 改进版2生成了{len(dialogue_list)}条对话")
    return dialogue_list

In [6]:
# Cell 5: 运行对比测试
print("=" * 80)
print("🧪 开始对比测试三种方法")
print("=" * 80)

# 测试原版
print("\n📍 测试原版方法...")
original_result = generate_dialogue_for_insertion_original(
    test_sentence_context, test_candidates, [], test_characters
)

print(f"\n📊 原版结果：{len(original_result)}轮对话")
for i, d in enumerate(original_result):
    print(f"  {i+1}. {d['speaker']}: {d['dialogue'][:50]}...")

# 测试改进版1
print("\n📍 测试改进版1（双重判断法）...")
v1_result = generate_dialogue_for_insertion_v1(
    test_sentence_context, test_candidates, [], test_characters
)

print(f"\n📊 改进版1结果：{len(v1_result)}轮对话")
for i, d in enumerate(v1_result):
    print(f"  {i+1}. {d['speaker']}: {d['dialogue'][:50]}...")

# 测试改进版2
print("\n📍 测试改进版2（简化判断法）...")
v2_result = generate_dialogue_for_insertion_v2(
    test_sentence_context, test_candidates, [], test_characters
)

print(f"\n📊 改进版2结果：{len(v2_result)}轮对话")
for i, d in enumerate(v2_result):
    print(f"  {i+1}. {d['speaker']}: {d['dialogue'][:50]}...")

🧪 开始对比测试三种方法

📍 测试原版方法...

🔍 原版：开始生成对话，候选角色: ['小红帽', '赛博狼']
原始 content: {"dialogue": "赛博狼，你听到了吗？星际森林好像在说话，机械残骸也在诉说着过去的故事。我们真的到达了一个全新的世界。", "action": "小红帽环顾四周，警觉地走下飞船舱门，试图辨认周围的声音和环境。"}...

  🔄 原版第1轮对话
原始 content: {"next_speaker": "赛博狼"}...
    📥 原版判断: 赛博狼
原始 content: {"dialogue": "小红帽，我的传感器检测到这里的能量波动异常，森林的低语中夹杂着未知的数据流，或许正是这个世界的语言。我们要小心前行，解读它隐藏的信息。", "action": "赛博狼调整耳部天线，启动环境分析模块，缓步走向星际森林的边缘。"}...

  🔄 原版第2轮对话
原始 content: {"next_speaker": "小红帽"}...
    📥 原版判断: 小红帽
原始 content: {"dialogue": "赛博狼，我们可以尝试同步你的数据解析器，让我也能听懂这些星际低语。也许森林会告诉我们如何继续前进。", "action": "小红帽轻轻触摸赛博狼的装甲接口，准备连接自己的通讯腕带，屏幕上开始浮现奇异的能量符号。"}...

  🔄 原版第3轮对话
原始 content: {"next_speaker": "赛博狼"}...
    📥 原版判断: 赛博狼
原始 content: {"dialogue": "小红帽，我正在将数据流译码并同步到你的感知装置，准备好接受信息输入。每一道低语都可能是藏宝图的线索，也可能是危机的警告。你感受到第一串星际密语了吗？", "action": "赛博狼连接小红帽头盔的数据端口，启动同步程序，星际低语化为可视化符号在小红帽的视野中浮现。"}...

  🔄 原版第4轮对话
原始 content: {"next_speaker": "小红帽"}...
    📥 原版判断: 小红帽
原始 content: {"dialogue": "赛博狼，我刚刚在脑海里听到了一句奇怪的星际密语：‘银色树影下，遗忘者的记忆指引方向。’你能定位那个‘银色树影’吗？", "action"

In [7]:
# Cell 6: 结果分析和选择
def analyze_results(original, v1, v2):
    """分析三种方法的效果"""
    print("\n📊 结果对比分析:")
    print(f"原版轮数: {len(original)}")
    print(f"改进版1轮数: {len(v1)}")
    print(f"改进版2轮数: {len(v2)}")
    
    print(f"\n🎯 推荐：")
    if len(v1) < len(original) and len(v1) > 0:
        print("✅ 改进版1效果最好 - 有目标导向且控制了长度")
    elif len(v2) < len(original) and len(v2) > 0:
        print("✅ 改进版2效果不错 - 简单但有效")
    else:
        print("⚠️ 需要进一步调整")

analyze_results(original_result, v1_result, v2_result)

# 选择最佳版本
best_version = None
if len(v1_result) <= 5 and len(v1_result) > 0:
    best_version = "v1"
    best_function = generate_dialogue_for_insertion_v1
elif len(v2_result) <= 5 and len(v2_result) > 0:
    best_version = "v2"
    best_function = generate_dialogue_for_insertion_v2
else:
    best_version = "original"
    best_function = generate_dialogue_for_insertion_original

print(f"\n🏆 最佳版本: {best_version}")


📊 结果对比分析:
原版轮数: 6
改进版1轮数: 2
改进版2轮数: 5

🎯 推荐：
✅ 改进版1效果最好 - 有目标导向且控制了长度

🏆 最佳版本: v1


In [8]:
# Cell 7: 生成最终可替换的函数
def generate_final_function():
    """生成最终可以替换到主程序的函数"""
    
    # 根据测试结果选择最佳版本
    if best_version == "v1":
        function_code = '''
def generate_dialogue_for_insertion(sentence_context, candidate_characters, full_plot, character_personality):
    """
    改进版：双重判断法，带目标导向的对话生成
    """
    print(f"\\n🔍 开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""

    # 第一步：分析对话目标
    goal_prompt = [{
        "role": "system",
        "content": f"""你是故事编剧。分析以下剧情片段需要用对话表达什么：

剧情：{sentence_context}
可用角色：{candidate_characters}

请分析：这段剧情最需要通过对话表达什么？预期几轮对话比较合适？

格式：{{"goal": "对话目标描述", "expected_rounds": 数字}}"""
    }]
    
    goal_response = generate_response(goal_prompt)
    goal_data = convert_json(goal_response)
    dialogue_goal = goal_data.get("goal", "推进剧情")
    expected_rounds = goal_data.get("expected_rounds", 3)

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，请基于以下剧情做出第一句发言。
剧情背景：{sentence_context}
对话目标：{dialogue_goal}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue": "...", "action": "..."}}"""
    }]
    
    response = generate_response(prompt_first)
    parsed = convert_json(response)
    
    if isinstance(parsed, dict) and "dialogue" in parsed:
        spoken_line = parsed.get("dialogue", "")
        action = parsed.get("action", "")
        dialogue_list.append({
            "speaker": speaker,
            "dialogue": spoken_line,
            "action": action
        })
        history += f"{speaker}: {spoken_line}\\n"

    # 多轮对话循环
    round_count = 0
    MAX_ROUNDS = 8
    while round_count < MAX_ROUNDS:
        round_count += 1
        
        # 双重判断逻辑
        judge_prompt = [{
            "role": "system",
            "content": f"""你是故事编剧。请分析当前对话状态：

【剧情背景】：{sentence_context}
【对话目标】：{dialogue_goal}
【预期轮数】：{expected_rounds}
【当前轮数】：{round_count}
【对话历史】：
{history}

请回答两个问题：
1. 对话目标是否已经达成？(0-10分评分)
2. 如果未充分达成，下一个发言人是谁？

可选角色：{candidate_characters}

格式：{{"goal_achieved": 分数, "should_continue": true/false, "next_speaker": "角色名或NONE", "reason": "判断理由"}}"""
        }]
        
        judge_res = generate_response(judge_prompt)
        judge_data = convert_json(judge_res)
        
        goal_achieved = judge_data.get("goal_achieved", 5)
        should_continue = judge_data.get("should_continue", True)
        next_speaker = judge_data.get("next_speaker", "NONE")
        
        # 停止条件
        if (not should_continue or 
            goal_achieved >= 7 or
            next_speaker == "NONE" or 
            next_speaker not in candidate_characters or
            round_count >= expected_rounds + 2):
            break

        # 生成发言内容
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
对话目标：{dialogue_goal}
对话历史：{history}
继续说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_reply)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": next_speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{next_speaker}: {spoken_line}\\n"
    
    print(f"  ✅ 生成了{len(dialogue_list)}条对话")
    return dialogue_list
'''
    elif best_version == "v2":
        function_code = '''
def generate_dialogue_for_insertion(sentence_context, candidate_characters, full_plot, character_personality):
    """
    改进版：简化判断法
    """
    print(f"\\n🔍 开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，基于剧情做出第一句发言：{sentence_context}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue": "...", "action": "..."}}"""
    }]
    
    response = generate_response(prompt_first)
    parsed = convert_json(response)
    
    if isinstance(parsed, dict) and "dialogue" in parsed:
        spoken_line = parsed.get("dialogue", "")
        action = parsed.get("action", "")
        dialogue_list.append({
            "speaker": speaker,
            "dialogue": spoken_line,
            "action": action
        })
        history += f"{speaker}: {spoken_line}\\n"

    # 简化判断循环
    round_count = 0
    while round_count < 6:
        round_count += 1
        
        # 简化判断
        judge_prompt = [{
            "role": "system",
            "content": f"""剧情：{sentence_context}
已有对话：{history}

这段对话是否已经足够表达剧情内容了？
- 如果已经足够，返回 "STOP"
- 如果还需要继续，从以下角色选择下一个发言人：{candidate_characters}

只返回角色名或"STOP"，不要解释。"""
        }]
        
        judge_res = generate_response(judge_prompt)
        next_speaker = judge_res.strip().strip('"').strip("'")
        
        if (next_speaker == "STOP" or 
            next_speaker not in candidate_characters or
            len(history.split('\\n')) >= 6):
            break

        # 生成发言
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
和对话历史：{history}
说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_reply)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": next_speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{next_speaker}: {spoken_line}\\n"
    
    print(f"  ✅ 生成了{len(dialogue_list)}条对话")
    return dialogue_list
'''
    else:
        function_code = "# 使用原版函数"
    
    print("🎯 最终可替换的函数代码：")
    print("=" * 60)
    print(function_code)
    print("=" * 60)
    
    return function_code

final_code = generate_final_function()

🎯 最终可替换的函数代码：

def generate_dialogue_for_insertion(sentence_context, candidate_characters, full_plot, character_personality):
    """
    改进版：双重判断法，带目标导向的对话生成
    """
    print(f"\n🔍 开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""

    # 第一步：分析对话目标
    goal_prompt = [{
        "role": "system",
        "content": f"""你是故事编剧。分析以下剧情片段需要用对话表达什么：

剧情：{sentence_context}
可用角色：{candidate_characters}

请分析：这段剧情最需要通过对话表达什么？预期几轮对话比较合适？

格式：{{"goal": "对话目标描述", "expected_rounds": 数字}}"""
    }]
    
    goal_response = generate_response(goal_prompt)
    goal_data = convert_json(goal_response)
    dialogue_goal = goal_data.get("goal", "推进剧情")
    expected_rounds = goal_data.get("expected_rounds", 3)

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，请基于以下剧情做出第一句发言。
剧情背景：{sentence_context}
对话目标：{dialogue_goal}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue":

✅ 成功导入项目模块
📁 使用输出目录: /Users/haha/Story/data/output/小红帽_科幻_nonlinear_T0.3_S3
📚 加载了story.json: 6 个章节
👥 加载了characters.json: 9 个角色
  - 小红帽: 无描述
  - 祖母: 无描述
  - 机械大灰狼: 无描述
  - 银河巡逻队队长: 无描述
  - 银河巡逻队队员甲: 无描述
  - 银河巡逻队队员乙: 无描述
  - 智能助手（小智）: 无描述
  - 虚拟野兽: 无描述
  - 未知幕后主使: 无描述

✅ 数据加载完成


In [6]:
# Cell 1: 环境设置和数据加载（更新版）
import json
import sys
import os
from pathlib import Path

# 设置项目路径
project_path = "/Users/haha/Story"
sys.path.append(project_path)

# 导入必要函数
try:
    from src.utils.utils import generate_response, convert_json, split_plot_into_sentences
    print("✅ 成功导入项目模块")
except ImportError as e:
    print(f"⚠️ 导入失败: {e}")
    # 如果导入失败，创建模拟函数
    def generate_response(messages):
        return '{"next_speaker": "NONE"}'
    def convert_json(text):
        try:
            return json.loads(text)
        except:
            return {}
    def split_plot_into_sentences(text):
        return text.split('。')

# 🔍 加载你的半成品数据
output_dir = Path("/Users/haha/Story/data/output/小红帽_科幻_nonlinear_T0.3_S3")

print(f"📁 使用输出目录: {output_dir}")

# 加载story.json
story_file = output_dir / "story.json"
if story_file.exists():
    with open(story_file, 'r', encoding='utf-8') as f:
        real_story = json.load(f)
    print(f"📚 加载了story.json: {len(real_story)} 个章节")
else:
    print("❌ 未找到story.json")
    real_story = []

# 加载characters.json（处理你的复杂格式）
characters_file = output_dir / "characters.json"
if characters_file.exists():
    with open(characters_file, 'r', encoding='utf-8') as f:
        real_characters = json.load(f)
    print(f"👥 加载了characters.json: {len(real_characters)} 个角色")
    
    # 展示角色信息
    for char in real_characters:
        name = char.get('name', '未知')
        role = char.get('role', '未知角色')
        traits = char.get('traits', '无描述')
        print(f"  - {name}: {role}")
        print(f"    特点: {traits[:50]}...")
    
    # 🎯 提取角色名列表，供对话生成使用
    character_names = [char.get('name', '') for char in real_characters if char.get('name')]
    print(f"\n📝 提取的角色名列表: {character_names}")
    
else:
    print("❌ 未找到characters.json")
    real_characters = []
    character_names = []

print("\n✅ 数据加载完成")

✅ 成功导入项目模块
📁 使用输出目录: /Users/haha/Story/data/output/小红帽_科幻_nonlinear_T0.3_S3
📚 加载了story.json: 6 个章节
👥 加载了characters.json: 7 个角色
  - 小红帽: 主角,星际任务执行者
    特点: 勇敢、机智、富有同情心、独立,有高度责任感...
  - 外婆: 任务目标,殖民星球居民
    特点: 慈祥、智慧、体弱多病、坚韧...
  - 大灰狼（狼型仿生体）: 主对手,芯片窃取者
    特点: 狡猾、冷静、善于伪装、有强大逻辑运算能力...
  - 母亲: 任务委托人,小红帽亲人
    特点: 关切、理智、坚强、善于谋划...
  - 人工智能助手（小白）: 辅助者,虚拟搭档
    特点: 智能、忠诚、幽默、反应迅速,善于分析和策略制定...
  - 银河巡逻队队长: 支援者,秩序维护者
    特点: 果断、正直、沉着、富有领导力...
  - 银河巡逻队成员: 支援团队,行动协助者
    特点: 专业、纪律严明、配合度高...

📝 提取的角色名列表: ['小红帽', '外婆', '大灰狼（狼型仿生体）', '母亲', '人工智能助手（小白）', '银河巡逻队队长', '银河巡逻队成员']

✅ 数据加载完成


In [11]:
# Cell 2: 智能角色选择函数
def select_relevant_characters(sentence, all_character_names, max_chars=3):
    """
    根据句子内容智能选择相关角色
    """
    relevant_chars = []
    
    # 先找句子中直接提到的角色
    for name in all_character_names:
        if name in sentence:
            relevant_chars.append(name)
    
    # 如果没有直接提到角色，根据关键词推断
    if not relevant_chars:
        if any(word in sentence for word in ['快递', '运送', '任务', '芯片']):
            if '小红帽' in all_character_names:
                relevant_chars.append('小红帽')
        
        if any(word in sentence for word in ['祖母', '奶奶', '老人']):
            if '祖母' in all_character_names:
                relevant_chars.append('祖母')
        
        if any(word in sentence for word in ['机械', '狼', '敌人', '攻击']):
            if '机械大灰狼' in all_character_names:
                relevant_chars.append('机械大灰狼')
        
        if any(word in sentence for word in ['助手', '智能', 'AI']):
            if '智能助手（小智）' in all_character_names:
                relevant_chars.append('智能助手（小智）')
    
    # 如果还是没有，默认选择主要角色
    if not relevant_chars:
        main_chars = ['小红帽', '祖母', '机械大灰狼']
        for char in main_chars:
            if char in all_character_names:
                relevant_chars.append(char)
                if len(relevant_chars) >= 2:
                    break
    
    # 限制角色数量
    return relevant_chars[:max_chars]

# 测试角色选择
test_sentence = "小红帽驾驶着快递飞船，准备将芯片送到祖母那里"
selected = select_relevant_characters(test_sentence, character_names)
print(f"🧪 测试句子: {test_sentence}")
print(f"🎯 选择的角色: {selected}")

🧪 测试句子: 小红帽驾驶着快递飞船，准备将芯片送到祖母那里
🎯 选择的角色: ['小红帽', '祖母']


In [12]:
# Cell 3: 从真实数据提取测试用例（更新版）
def extract_test_cases_from_story_updated(story_data, character_names, max_cases=3):
    """从你的real story中提取测试用例 - 处理复杂格式"""
    test_cases = []
    
    for i, chapter in enumerate(story_data[:max_cases]):
        plot = chapter.get("plot", "")
        chapter_characters = chapter.get("characters", [])
        
        if plot:
            # 取plot的前两句作为测试
            sentences = split_plot_into_sentences(plot)
            if len(sentences) >= 2:
                test_sentence = sentences[0] + "。" + sentences[1] + "。"
            else:
                test_sentence = plot[:200] + "。"  # 如果句子不够，取前200字符
            
            # 🎯 智能选择相关角色
            relevant_characters = select_relevant_characters(test_sentence, character_names, max_chars=3)
            
            # 如果智能选择失败，尝试从章节中提取
            if not relevant_characters and chapter_characters:
                for char_info in chapter_characters:
                    if isinstance(char_info, dict):
                        name = char_info.get("name", "")
                        if name and name in character_names:
                            relevant_characters.append(name)
                    elif isinstance(char_info, str) and char_info in character_names:
                        relevant_characters.append(char_info)
                
                relevant_characters = relevant_characters[:3]
            
            # 确保至少有2个角色
            if len(relevant_characters) >= 2:
                test_cases.append({
                    "chapter_id": chapter.get("chapter_id", f"Chapter_{i+1}"),
                    "title": chapter.get("title", "未知标题"),
                    "sentence": test_sentence,
                    "characters": relevant_characters,
                    "scene": chapter.get("scene", ""),
                    "original_plot": plot[:100] + "..."  # 保存原始plot片段用于参考
                })
    
    return test_cases

# 提取测试用例
if real_story and character_names:
    test_cases = extract_test_cases_from_story_updated(real_story, character_names, max_cases=3)
    print(f"🧪 从你的story.json中提取了 {len(test_cases)} 个测试用例:")
    for i, case in enumerate(test_cases):
        print(f"\n  {i+1}. {case['chapter_id']}: {case['title']}")
        print(f"     测试句子: {case['sentence'][:80]}...")
        print(f"     选择角色: {case['characters']}")
        print(f"     场景: {case['scene'][:60]}...")
else:
    print("❌ 没有story数据或角色数据，无法提取测试用例")
    test_cases = []

🧪 从你的story.json中提取了 1 个测试用例:

  1. Chapter 1: 星际小红帽的任务
     测试句子: 小红帽紧盯着导航终端上的航线,内心充满着焦虑与希望.。她的思绪时常被回忆牵引,浮现出祖母慈祥的面容——在偏远太空站里,那位年迈却乐观的亲人,曾用颤抖的手抚摸她的...
     选择角色: ['小红帽', '祖母']
     场景: 星际快递飞船“赤焰号”内,小红帽独自驾驶着飞船穿梭于点缀着星云与小行星的幽暗宇宙航道.飞船舱壁上映出明灭的仪表灯,外部舷...


In [14]:
# Cell 4: 定义改进版对话生成函数（适配你的角色格式）
def generate_dialogue_improved_for_your_data(sentence_context, candidate_characters, full_plot, character_personality):
    """
    改进版对话生成 - 专门适配你的角色数据格式
    """
    print(f"\n🔍 改进版：开始生成对话，候选角色: {candidate_characters}")
    
    dialogue_list = []
    history = ""

    # 🎯 利用你的详细角色信息
    character_info = {}
    for char in character_personality:
        name = char.get('name', '')
        if name in candidate_characters:
            character_info[name] = {
                'role': char.get('role', ''),
                'traits': char.get('traits', ''),
                'background': char.get('background', ''),
                'motivation': char.get('motivation', '')
            }
    
    print(f"  📋 角色信息: {list(character_info.keys())}")

    # 分析对话目标 - 结合角色背景
    goal_prompt = [{
        "role": "system",
        "content": f"""你是故事编剧。分析以下剧情片段需要用对话表达什么：

剧情：{sentence_context}
可用角色：{candidate_characters}

角色背景信息：
{json.dumps(character_info, ensure_ascii=False, indent=2)}

请分析：这段剧情最需要通过对话表达什么？预期几轮对话比较合适？

⚠️ 重要：expected_rounds必须是单个整数（如2、3、4），不能是范围

格式：{{"goal": "对话目标描述", "expected_rounds": 整数}}"""
    }]
    
    try:
        goal_response = generate_response(goal_prompt)
        goal_data = convert_json(goal_response)
        
        if isinstance(goal_data, dict):
            dialogue_goal = goal_data.get("goal", "推进剧情")
            expected_rounds = goal_data.get("expected_rounds", 3)
            
            try:
                expected_rounds = int(expected_rounds)
            except (ValueError, TypeError):
                expected_rounds = 3
        else:
            dialogue_goal = "推进剧情"
            expected_rounds = 3
            
    except Exception as e:
        print(f"⚠️ 目标分析失败: {e}")
        dialogue_goal = "推进剧情"
        expected_rounds = 3

    print(f"  🎯 对话目标: {dialogue_goal}")
    print(f"  📊 预期轮数: {expected_rounds}")

    # 第一个发言人 - 结合角色性格
    speaker = candidate_characters[0]
    speaker_info = character_info.get(speaker, {})
    
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}。

角色信息：
- 角色定位：{speaker_info.get('role', '未知')}
- 性格特点：{speaker_info.get('traits', '未知')}
- 背景：{speaker_info.get('background', '未知')}
- 动机：{speaker_info.get('motivation', '未知')}

请基于以下剧情做出第一句发言：
剧情背景：{sentence_context}
对话目标：{dialogue_goal}
其他角色：{[c for c in candidate_characters if c != speaker]}

请说出符合你性格特点和动机的话，格式：
{{"dialogue": "...", "action": "..."}}"""
    }]
    
    try:
        response = generate_response(prompt_first)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{speaker}: {spoken_line}\n"
    except Exception as e:
        print(f"⚠️ 第一句对话生成失败: {e}")
        return []

    # 多轮对话循环
    round_count = 0
    SAFETY_LIMIT = 10
    
    while round_count < SAFETY_LIMIT:
        round_count += 1
        print(f"\n  🔄 改进版第{round_count}轮对话")
        
        # 判断是否继续
        judge_prompt = [{
            "role": "system",
            "content": f"""你是故事编剧。请分析当前对话状态：

【剧情背景】：{sentence_context}
【对话目标】：{dialogue_goal}
【预期轮数】：{expected_rounds}
【当前轮数】：{round_count}
【对话历史】：
{history}

【角色信息】：
{json.dumps(character_info, ensure_ascii=False, indent=2)}

请判断：
1. 对话目标是否已经达成？(0-10分评分)
2. 是否应该继续对话？
3. 如果继续，哪个角色最适合说下一句？

可选角色：{candidate_characters}

格式：{{"goal_achieved": 分数, "should_continue": true/false, "next_speaker": "角色名或NONE", "reason": "判断理由"}}"""
        }]
        
        try:
            judge_res = generate_response(judge_prompt)
            judge_data = convert_json(judge_res)
            
            if not isinstance(judge_data, dict):
                print(f"⚠️ 判断返回非dict格式")
                break
                
            goal_achieved = judge_data.get("goal_achieved", 5)
            should_continue = judge_data.get("should_continue", False)
            next_speaker = judge_data.get("next_speaker", "NONE")
            reason = judge_data.get("reason", "")
            
            print(f"    📊 目标达成度: {goal_achieved}/10")
            print(f"    🤔 是否继续: {should_continue}")
            print(f"    👤 下一发言人: {next_speaker}")
                
        except Exception as e:
            print(f"⚠️ 对话判断失败: {e}")
            break
        
        # 停止条件
        if (not should_continue or
            next_speaker == "NONE" or 
            next_speaker not in candidate_characters):
            print(f"  🛑 改进版结束对话")
            break

        # 生成下一句对话
        next_speaker_info = character_info.get(next_speaker, {})
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}。

角色信息：
- 角色定位：{next_speaker_info.get('role', '未知')}
- 性格特点：{next_speaker_info.get('traits', '未知')}
- 背景：{next_speaker_info.get('background', '未知')}
- 动机：{next_speaker_info.get('motivation', '未知')}

基于以下情况说一句话：
剧情：{sentence_context}
对话目标：{dialogue_goal}
对话历史：{history}

请说出符合你性格和动机的话，格式：
{{"dialogue": "...", "action": "..."}}"""
        }]
        
        try:
            response = generate_response(prompt_reply)
            parsed = convert_json(response)
            
            if isinstance(parsed, dict) and "dialogue" in parsed:
                spoken_line = parsed.get("dialogue", "")
                action = parsed.get("action", "")
                dialogue_list.append({
                    "speaker": next_speaker,
                    "dialogue": spoken_line,
                    "action": action
                })
                history += f"{next_speaker}: {spoken_line}\n"
        except Exception as e:
            print(f"⚠️ 发言生成失败: {e}")
            continue
    
    if round_count >= SAFETY_LIMIT:
        print(f"⚠️ 达到安全保底({SAFETY_LIMIT}轮)")
    
    print(f"  ✅ 改进版生成了{len(dialogue_list)}条对话")
    return dialogue_list

In [15]:
# Cell 5: 运行针对你数据的专门测试
def run_your_data_test():
    """专门针对你的数据格式运行测试"""
    
    if not test_cases:
        print("❌ 没有可用的测试用例")
        return []
    
    results = []
    
    for i, case in enumerate(test_cases):
        print(f"\n{'='*80}")
        print(f"🧪 测试用例 {i+1}: {case['chapter_id']}")
        print(f"📖 标题: {case['title']}")
        print(f"📝 测试句子: {case['sentence']}")
        print(f"👥 选择角色: {case['characters']}")
        print(f"🎬 场景: {case['scene']}")
        print('='*80)
        
        # 只测试改进版，因为重点是验证新函数
        print(f"\n📍 测试改进版方法...")
        try:
            improved_result = generate_dialogue_improved_for_your_data(
                case['sentence'], 
                case['characters'], 
                [], 
                real_characters  # 传入完整的角色信息
            )
            print(f"✅ 改进版完成: {len(improved_result)}轮对话")
            
            # 展示生成的对话
            print(f"\n💬 生成的对话:")
            for j, dialogue in enumerate(improved_result):
                print(f"  {j+1}. {dialogue['speaker']}: {dialogue['dialogue']}")
                if dialogue['action']:
                    print(f"     动作: {dialogue['action']}")
            
        except Exception as e:
            print(f"❌ 改进版失败: {e}")
            import traceback
            traceback.print_exc()
            improved_result = []
        
        # 记录结果
        result = {
            "case": case,
            "improved_rounds": len(improved_result),
            "improved_dialogue": improved_result,
            "success": len(improved_result) > 0
        }
        results.append(result)
        
        print(f"\n📊 本轮结果: {len(improved_result)}轮对话")
    
    return results

# 运行你的专门测试
if test_cases:
    print("🚀 开始使用你的真实数据测试改进版对话生成...")
    your_test_results = run_your_data_test()
else:
    print("⚠️ 没有测试用例，跳过测试")
    your_test_results = []

🚀 开始使用你的真实数据测试改进版对话生成...

🧪 测试用例 1: Chapter 1
📖 标题: 星际小红帽的任务
📝 测试句子: 小红帽紧盯着导航终端上的航线,内心充满着焦虑与希望.。她的思绪时常被回忆牵引,浮现出祖母慈祥的面容——在偏远太空站里,那位年迈却乐观的亲人,曾用颤抖的手抚摸她的发梢,鼓励她勇敢追梦.。
👥 选择角色: ['小红帽', '祖母']
🎬 场景: 星际快递飞船“赤焰号”内,小红帽独自驾驶着飞船穿梭于点缀着星云与小行星的幽暗宇宙航道.飞船舱壁上映出明灭的仪表灯,外部舷窗投下深蓝与紫色交织的星河光芒,舱内空间虽狭小却井然有序,一只红色披风随小红帽的动作在座椅边垂下.仪表盘上,医疗纳米芯片安静地封存于全息防护盒中,散发着微微蓝光.

📍 测试改进版方法...

🔍 改进版：开始生成对话，候选角色: ['小红帽', '祖母']
  📋 角色信息: ['小红帽', '祖母']
原始 content: {"goal": "通过对话展现小红帽对祖母的思念、担忧与希望，以及祖母对小红帽的鼓励和爱，强化祖孙间深厚情感，为小红帽冒险动机提供情感铺垫。", "expected_rounds": 2}...
  🎯 对话目标: 通过对话展现小红帽对祖母的思念、担忧与希望,以及祖母对小红帽的鼓励和爱,强化祖孙间深厚情感,为小红帽冒险动机提供情感铺垫.
  📊 预期轮数: 2
原始 content: {"dialogue": "导航说还要两个星系才能到达，但我真的好担心……奶奶，您一定要等我啊，我带着最好的医疗纳米芯片来了！", "action": "小红帽紧紧握住怀里的快递盒，眼中浮现出祖母温柔的笑容，轻声呢喃，仿佛祖母就在身边。"}...

  🔄 改进版第1轮对话
原始 content: {"goal_achieved": 5, "should_continue": true, "next_speaker": "祖母", "reason": "小红帽已经表达了对祖母的思念和担忧，并展现了希望，但祖母的回应尚未出现，祖孙间的深厚情感还未完全展现。需要祖母回应，给予鼓励和爱，进一步强化情感，为小红帽的冒险动机做更充分的铺垫。"}...
    📊 目标达成度: 5/10
    🤔 是否继续: True
    👤 下一发言人: 祖母
原始 content: {

In [17]:
# Cell 6: 分析你的测试结果
def analyze_your_test_results(results):
    """分析你的测试结果"""
    if not results:
        print("❌ 没有测试结果可分析")
        return False
    
    print(f"\n{'='*60}")
    print("📊 你的真实数据测试结果")
    print('='*60)
    
    success_count = sum(1 for r in results if r['success'])
    total_count = len(results)
    average_rounds = sum(r['improved_rounds'] for r in results) / len(results) if results else 0
    
    print(f"📈 基本统计:")
    print(f"   测试用例总数: {total_count}")
    print(f"   成功生成对话: {success_count}")
    print(f"   成功率: {success_count/total_count*100:.1f}%")
    print(f"   平均对话轮数: {average_rounds:.1f}轮")
    
    print(f"\n📝 详细结果:")
    for i, result in enumerate(results):
        case = result["case"]
        print(f"\n{i+1}. {case['chapter_id']}: {case['title']}")
        print(f"   对话轮数: {result['improved_rounds']}")
        print(f"   角色组合: {case['characters']}")
        
        if result["improved_dialogue"]:
            print("   对话示例:")
            for j, d in enumerate(result["improved_dialogue"][:2]):
                print(f"     {j+1}. {d['speaker']}: {d['dialogue'][:60]}...")
    
    # 评估结果
    if average_rounds <= 0:
        quality = "❌ 失败"
        recommendation = "函数存在严重问题，需要调试"
        should_replace = False
    elif average_rounds > 8:
        quality = "⚠️ 过长"
        recommendation = "对话轮数偏多，需要调整判断逻辑"
        should_replace = False
    elif 3 <= average_rounds <= 6:
        quality = "✅ 理想"
        recommendation = "对话长度合理，可以替换到主程序"
        should_replace = True
    elif average_rounds < 3:
        quality = "⚠️ 过短"
        recommendation = "对话可能过于简短，但比无限循环好"
        should_replace = True
    else:
        quality = "👍 良好"
        recommendation = "效果不错，建议替换"
        should_replace = True
    
    print(f"\n🎯 最终评估:")
    print(f"   质量评级: {quality}")
    print(f"   平均轮数: {average_rounds:.1f}轮")
    print(f"   推荐操作: {recommendation}")
    
    return should_replace

# 分析你的结果
if your_test_results:
    should_replace = analyze_your_test_results(your_test_results)
    
    if should_replace:
        print(f"\n🎉 测试通过！可以安全替换到主程序")
        print(f"\n📋 下一步操作:")
        print("1. 将Cell 4中的函数重命名为 generate_dialogue_for_insertion")
        print("2. 复制到 /Users/haha/Story/src/generation/dialogue_inserter.py")
        print("3. 运行完整流程测试")
    else:
        print(f"\n🤔 需要进一步调试和优化")
else:
    print("⚠️ 请先运行测试")


📊 你的真实数据测试结果
📈 基本统计:
   测试用例总数: 1
   成功生成对话: 1
   成功率: 100.0%
   平均对话轮数: 3.0轮

📝 详细结果:

1. Chapter 1: 星际小红帽的任务
   对话轮数: 3
   角色组合: ['小红帽', '祖母']
   对话示例:
     1. 小红帽: 导航说还要两个星系才能到达,但我真的好担心……奶奶,您一定要等我啊,我带着最好的医疗纳米芯片来了！...
     2. 祖母: 傻孩子,别担心奶奶,奶奶一直在等你呢.你能这么勇敢,奶奶已经很骄傲了.旅途漫长,你也要照顾好自己,等你平安到来,奶奶一定...

🎯 最终评估:
   质量评级: ✅ 理想
   平均轮数: 3.0轮
   推荐操作: 对话长度合理，可以替换到主程序

🎉 测试通过！可以安全替换到主程序

📋 下一步操作:
1. 将Cell 4中的函数重命名为 generate_dialogue_for_insertion
2. 复制到 /Users/haha/Story/src/generation/dialogue_inserter.py
3. 运行完整流程测试


In [3]:
# Cell: 重新加载修改后的模块
import importlib
import sys

# 重新导入你修改的模块
sys.path.append('/Users/haha/Story')
from src.generation import dialogue_inserter
importlib.reload(dialogue_inserter)  # 重新加载修改后的代码

print("✅ 重新加载dialogue_inserter模块完成")

✅ 重新加载dialogue_inserter模块完成


In [4]:
# Cell: 快速测试修改后的函数
from src.generation.dialogue_inserter import generate_dialogue_for_insertion

# 使用你已有的test_cases
if test_cases:
    test_case = test_cases[0]  # 取第一个测试用例
    print(f"🧪 测试: {test_case['title']}")
    print(f"📝 句子: {test_case['sentence'][:100]}...")
    
    result = generate_dialogue_for_insertion(
        test_case['sentence'],
        test_case['characters'],
        [],
        real_characters
    )
    
    print(f"📊 结果: {len(result)}轮对话")
    for i, d in enumerate(result):
        print(f"  {i+1}. {d['speaker']}: {d['dialogue'][:60]}...")

NameError: name 'test_cases' is not defined

In [20]:
# Cell: 对比测试
def test_before_after(sentence, characters):
    print("📊 对比测试:")
    
    # 如果你保存了旧版本函数
    old_result = generate_dialogue_for_insertion_original(sentence, characters, [], real_characters)
    new_result = generate_dialogue_for_insertion(sentence, characters, [], real_characters)
    
    print(f"旧版本: {len(old_result)}轮")
    print(f"新版本: {len(new_result)}轮")
    print(f"差异: {len(new_result) - len(old_result)}轮")
    
    return old_result, new_result

# 运行对比
if test_cases:
    old, new = test_before_after(test_cases[0]['sentence'], test_cases[0]['characters'])

📊 对比测试:

🔍 原版：开始生成对话，候选角色: ['小红帽', '祖母']
原始 content: {"dialogue": "祖母，我快要到你那里了，您还好吗？我一直记着您说的话，努力勇敢地面对每一个未知。", "action": "小红帽用手轻抚着导航终端，语气中带着一丝颤抖和期待，试图通过通讯设备联系祖母。"}...

  🔄 原版第1轮对话
原始 content: {"next_speaker": "祖母"}...
    📥 原版判断: 祖母
原始 content: {"dialogue": "亲爱的红帽，我一直在太空站的窗前盼着你呢。你这么勇敢，祖母为你骄傲！记得，不论飞船遇到什么状况，都要相信自己，也要随时注意安全哦。", "action": "祖母轻轻地将手贴在通讯屏幕上，仿佛想要穿越宇宙拥抱小红帽。"}...

  🔄 原版第2轮对话
原始 content: {"next_speaker": "小红帽"}...
    📥 原版判断: 小红帽
原始 content: {"dialogue": "谢谢您，祖母。您的鼓励一直是我前进的动力。我会小心导航，每一步都不会辜负您的期望。很快就能见到您了！", "action": "小红帽调整飞船航向，同时再次检查生命维持系统，确保安全抵达太空站。"}...

  🔄 原版第3轮对话
原始 content: {"next_speaker": "祖母"}...
    📥 原版判断: 祖母
原始 content: {"dialogue": "红帽呀，路途遥远，记得时常补充能量，别让自己太累了。祖母会在太空站的灯下等你归来，等着给你一个温暖的拥抱。", "action": "祖母用手轻抚着窗台上的旧照片，目光温柔地望向星空，一边为红帽准备她最爱的小点心。"}...

  🔄 原版第4轮对话
原始 content: {"next_speaker": "小红帽"}...
    📥 原版判断: 小红帽
原始 content: {"dialogue": "祖母，您的话让我感觉像有一束温暖的光一直在指引着我。等见到您，我一定要把旅途中收集到的故事都讲给您听！", "action": "小红帽轻轻抚摸着终端旁的护身符，深吸一口气，调整飞船能量系统，继续沿着导航航线前进。"}...

  🔄 原版第5轮对话
原始 con

In [12]:
# 重新加载并测试完整的对话生成流程
import importlib
import sys
sys.path.append('/Users/haha/Story')

# 重新导入修改后的模块
from src.generation import dialogue_inserter
importlib.reload(dialogue_inserter)

# 测试analyze_dialogue_insertions_v2（这是main_pipeline实际调用的函数）
print("🧪 测试完整的analyze_dialogue_insertions_v2流程...")

# 使用你的real_story前两个章节
test_chapters = real_story[:2] if len(real_story) >= 2 else real_story

chapter_results, sentence_results, behavior_timeline = dialogue_inserter.analyze_dialogue_insertions_v2(
    test_chapters, 
    real_characters
)

print(f"📊 完整流程结果:")
print(f"   章节结果: {len(chapter_results)} 个")
print(f"   句子结果: {len(sentence_results)} 个")
print(f"   行为时间线: {len(behavior_timeline)} 条")

🧪 测试完整的analyze_dialogue_insertions_v2流程...
章节Chapter 5分割为8个句子
原始 content: [
  {"sentence":"小红帽首次踏上银河救援飞船,脚步略显急促.", "need_to_action":0, "actor_list":[]},
  {"sentence":"她握着医疗芯片,目光凝视着舷窗外的未知星域.", "need_to_action":0, "actor_list":[]},
  {"sentence":"舱内气氛紧张,仪表盘不断跳动,显示着殖民地的危急信号.", "need_to_action":0, "actor_list":[]},
  {"sentence":"这时,银河巡逻队队长带着冷静的面容走进舱室,身后是全副武装的队员.", "need_to_action":0, "actor_list":[]},
  {"sentence":"他们相互打量,言语谨慎,彼此都在揣测对方的能力与意图.", "need_to_action":1, "actor_list":["小红帽", "银河巡逻队队长", "银河巡逻队成员"]},
  {"sentence":"队长简要说明任务安排,声音低沉却不失威严,而小红帽则以坚定的语气回应,隐约流露出责任感与不安.", "need_to_action":1, "actor_list":["银河巡逻队队长", "小红帽"]},
  {"sentence":"两支队伍间的隔阂与陌生感在警报声中愈发明显.", "need_to_action":0, "actor_list":[]},
  {"sentence":"飞船即将穿越防御网,救援行动一触即发,空气中弥漫着未知的紧张与希望.", "need_to_action":0, "actor_list":[]}
]...

🔍 开始生成对话，候选角色: ['小红帽', '银河巡逻队队长', '银河巡逻队成员']
原始 content: {"goal": "通过对话展现角色之间的试探与防备，表达他们对彼此身份、能力和真实意图的怀疑和推敲，从而营造悬疑和紧张感。", "expected_rounds": 3}...
  🎯 目标分析返回: 3
原始 content: {"dialogue": "你们来自银河巡逻

In [13]:
# 分析生成的对话质量
def analyze_dialogue_quality(sentence_results):
    """分析sentence_results中的对话质量"""
    
    total_sentences = len(sentence_results)
    dialogue_sentences = [s for s in sentence_results if s.get('need_to_action') == 1]
    no_dialogue_sentences = [s for s in sentence_results if s.get('need_to_action') == 0]
    
    print(f"📊 对话分布分析:")
    print(f"   总句子数: {total_sentences}")
    print(f"   有对话句子: {len(dialogue_sentences)} ({len(dialogue_sentences)/total_sentences*100:.1f}%)")
    print(f"   无对话句子: {len(no_dialogue_sentences)} ({len(no_dialogue_sentences)/total_sentences*100:.1f}%)")
    
    # 分析对话轮数分布
    rounds_distribution = {}
    for s in dialogue_sentences:
        rounds = len(s.get('dialogue', []))
        rounds_distribution[rounds] = rounds_distribution.get(rounds, 0) + 1
    
    print(f"\n🔄 对话轮数分布:")
    for rounds, count in sorted(rounds_distribution.items()):
        print(f"   {rounds}轮: {count}次")
    
    # 计算平均轮数
    if dialogue_sentences:
        avg_rounds = sum(len(s.get('dialogue', [])) for s in dialogue_sentences) / len(dialogue_sentences)
        print(f"   平均轮数: {avg_rounds:.1f}轮")
    
    return dialogue_sentences, rounds_distribution

# 运行质量分析
dialogue_sentences, rounds_dist = analyze_dialogue_quality(sentence_results)

📊 对话分布分析:
   总句子数: 17
   有对话句子: 6 (35.3%)
   无对话句子: 11 (64.7%)

🔄 对话轮数分布:
   2轮: 1次
   3轮: 1次
   5轮: 1次
   6轮: 1次
   19轮: 2次
   平均轮数: 9.0轮


In [14]:
# 检查rounds_statistics.jsonl文件
import json
from pathlib import Path

stats_file = Path("/Users/haha/Story/data/output/logs/rounds_statistics.jsonl")
print(f"📊 检查统计文件: {stats_file}")

if stats_file.exists():
    with open(stats_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    print(f"📈 已记录 {len(lines)} 条对话统计")
    
    # 解析最近的统计数据
    recent_stats = []
    for line in lines[-10:]:  # 看最后10条
        try:
            data = json.loads(line)
            recent_stats.append(data)
        except:
            continue
    
    if recent_stats:
        print(f"\n📋 最近的统计数据:")
        for i, stat in enumerate(recent_stats):
            expected = stat.get('expected_rounds', 0)
            actual = stat.get('actual_rounds', 0)
            deviation = stat.get('deviation', 0)
            context = stat.get('sentence_context', '')[:50]
            characters = stat.get('characters', [])
            
            print(f"   {i+1}. 预期{expected}轮 → 实际{actual}轮 (偏差{deviation:+d})")
            print(f"      角色: {characters}")
            print(f"      句子: {context}...")
        
        # 计算统计
        avg_expected = sum(s.get('expected_rounds', 0) for s in recent_stats) / len(recent_stats)
        avg_actual = sum(s.get('actual_rounds', 0) for s in recent_stats) / len(recent_stats)
        avg_deviation = sum(s.get('deviation', 0) for s in recent_stats) / len(recent_stats)
        
        print(f"\n📊 统计汇总:")
        print(f"   平均预期轮数: {avg_expected:.1f}")
        print(f"   平均实际轮数: {avg_actual:.1f}")
        print(f"   平均偏差: {avg_deviation:+.1f}")
else:
    print("❌ 统计文件不存在")

📊 检查统计文件: /Users/haha/Story/data/output/logs/rounds_statistics.jsonl
📈 已记录 50 条对话统计

📋 最近的统计数据:
   1. 预期3轮 → 实际11轮 (偏差+8)
      角色: ['机械狼', '小红帽']
      句子: 突然,主控区的系统遭受黑客入侵,警报骤响,一只银色机械狼从阴影中现身,眼中闪烁着红色电流,试图通过变...
   2. 预期3轮 → 实际13轮 (偏差+10)
      角色: ['小红帽', '空间站AI管家']
      句子: 小红帽依靠自己的科技知识,迅速唤醒空间站AI管家的防御系统,设下诱导陷阱....
   3. 预期3轮 → 实际6轮 (偏差+3)
      角色: ['机械狼', '小红帽', '空间站AI管家']
      句子: 机械狼在虚拟空间与小红帽展开智斗,数据流与激光在空中交错,机械狼不断尝试破解防护,但小红帽巧妙利用环...
   4. 预期3轮 → 实际2轮 (偏差-1)
      角色: ['小红帽', '外婆']
      句子: 小红帽将医疗芯片郑重地交到外婆手中,母女重逢的温情与守护空间站的紧迫感交错,星际英雄归来,守护者间的...
   5. 预期3轮 → 实际19轮 (偏差+16)
      角色: ['小红帽', '银河巡逻队队长', '银河巡逻队成员']
      句子: 他们相互打量,言语谨慎,彼此都在揣测对方的能力与意图....
   6. 预期2轮 → 实际2轮 (偏差+0)
      角色: ['银河巡逻队队长', '小红帽']
      句子: 队长简要说明任务安排,声音低沉却不失威严,而小红帽则以坚定的语气回应,隐约流露出责任感与不安....
   7. 预期2轮 → 实际5轮 (偏差+3)
      角色: ['小白', '小红帽']
      句子: 耳边,小白的虚拟声音低语,分析前方能量波动与生物活动数据....
   8. 预期3轮 → 实际19轮 (偏差+16)
      角色: ['小白', '小红帽']
      句子: 每当小红帽遇到错综复杂的能量屏障或疑似陷阱,小白便迅速推送破解指令,协助她灵巧穿越....
   9. 预期3轮 → 实际3轮 (偏差+

In [11]:
# Cell: 在ipynb中测试多模型验证的完整代码
from datetime import datetime
import json

def external_dialogue_judge_test(dialogue_list, sentence_context):
    """独立的对话质量判断者 - 使用Claude Sonnet 4"""
    
    if not dialogue_list:
        return {"should_stop": False, "reason": "对话为空", "quality_score": 5}
    
    dialogue_text = "\n".join([f"{d['speaker']}: {d['dialogue']}" for d in dialogue_list])
    current_rounds = len(dialogue_list)
    
    judge_prompt = [{
        "role": "system", 
        "content": f"""你是对话质量评估专家。评估以下对话是否应该停止：

原始剧情句子：{sentence_context}
当前轮数：{current_rounds}

生成的对话：
{dialogue_text}

评估标准：
1. 重复循环检测：是否出现明显的重复模式或循环对话
2. 剧情偏离：是否过度脱离了原始剧情句子
3. 信息密度：最近几轮是否信息量过低，只是在重复确认
4. 自然完整性：对话是否已经表达完了这个句子该表达的内容

质量评分：
- 1-3分：明显有问题，应立即停止
- 4-6分：一般质量，可继续但需警惕
- 7-10分：质量良好，可继续

只返回以下格式的JSON：{{"should_stop": true/false, "reason": "具体原因", "quality_score": 1-10的数字}}
不要添加任何解释。"""
    }]
    
    try:
        # 使用Claude Sonnet 4模型进行判断
        response = generate_response(
            judge_prompt, 
            model="claude-sonnet-4-20250514",
            temperature=0.3  # 判断任务用较低temperature
        )
        judge_result = convert_json(response)
        
        # 强化返回值检查
        if isinstance(judge_result, dict):
            should_stop = judge_result.get("should_stop", False)
            reason = judge_result.get("reason", "")
            quality_score = judge_result.get("quality_score", 5)
            
            # 确保quality_score是数字
            try:
                quality_score = int(quality_score)
            except (ValueError, TypeError):
                quality_score = 5
                
            return {
                "should_stop": should_stop,
                "reason": reason,
                "quality_score": quality_score
            }
        else:
            return {"should_stop": False, "reason": "判断失败", "quality_score": 5}
            
    except Exception as e:
        print(f"⚠️ 外部判断失败: {e}")
        return {"should_stop": False, "reason": "判断失败", "quality_score": 5}


def generate_dialogue_test_with_external_judge(sentence_context, candidate_characters):
    """测试版本：带多模型验证的对话生成"""
    
    session_id = datetime.now().strftime("%H%M%S")
    print(f"\n🔍 开始生成对话，候选角色: {candidate_characters}")
    print(f"📝 测试句子: {sentence_context[:60]}...")
    
    dialogue_list = []
    history = ""
    
    # 分析对话目标（仍然记录expected_rounds用于统计）
    goal_prompt = [{
        "role": "system",
        "content": f"""你是故事编剧。分析以下剧情片段需要用对话表达什么：

剧情：{sentence_context}
可用角色：{candidate_characters}

请分析：这段剧情最需要通过对话表达什么？预期几轮对话比较合适？

⚠️ 重要：expected_rounds必须是单个整数（如2、3、4），不能是范围

格式：{{"goal": "对话目标描述", "expected_rounds": 整数}}"""
    }]
    
    try:
        goal_response = generate_response(goal_prompt)
        goal_data = convert_json(goal_response)
        
        if isinstance(goal_data, dict):
            dialogue_goal = goal_data.get("goal", "推进剧情")
            expected_rounds = goal_data.get("expected_rounds", 3)
            try:
                expected_rounds = int(expected_rounds)
            except (ValueError, TypeError):
                expected_rounds = 3
        else:
            dialogue_goal = "推进剧情"
            expected_rounds = 3
    except Exception as e:
        print(f"⚠️ 目标分析失败: {e}")
        dialogue_goal = "推进剧情"
        expected_rounds = 3

    print(f"🎯 对话目标: {dialogue_goal}")
    print(f"📊 预期轮数: {expected_rounds}")

    # 第一个发言人
    speaker = candidate_characters[0]
    prompt_first = [{
        "role": "system",
        "content": f"""你是 {speaker}，请基于以下剧情做出第一句发言。
剧情背景：{sentence_context}
对话目标：{dialogue_goal}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue": "...", "action": "..."}}"""
    }]
    
    try:
        response = generate_response(prompt_first)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            spoken_line = parsed.get("dialogue", "")
            action = parsed.get("action", "")
            dialogue_list.append({
                "speaker": speaker,
                "dialogue": spoken_line,
                "action": action
            })
            history += f"{speaker}: {spoken_line}\n"
            print(f"  1. {speaker}: {spoken_line[:60]}...")
    except Exception as e:
        print(f"⚠️ 第一句对话生成失败: {e}")
        return [], expected_rounds, 0

    # 循环判断（加入多模型验证）
    round_count = 0
    SAFETY_LIMIT = 15
    external_judge_calls = 0
    
    while round_count < SAFETY_LIMIT:
        round_count += 1
        
        # 每3轮进行一次外部质量检查
        if round_count >= 3 and round_count % 3 == 0:
            external_judge_calls += 1
            print(f"    🔍 第{external_judge_calls}次外部质量检查（第{round_count}轮）")
            
            judge_result = external_dialogue_judge_test(dialogue_list, sentence_context)
            quality_score = judge_result.get("quality_score", 5)
            should_stop = judge_result.get("should_stop", False)
            reason = judge_result.get("reason", "")
            
            print(f"    📊 Claude外部判断: quality_score={quality_score}/10, should_stop={should_stop}")
            print(f"    💭 理由: {reason}")
            
            # 质量过低时强制停止
            if should_stop and quality_score <= 4:
                print(f"  🛑 Claude外部判断建议停止（质量分数:{quality_score}）")
                break

        # 原有的LLM自我判断（不包含预期轮数信息）
        judge_prompt = [{
            "role": "system",
            "content": f"""你是故事编剧。分析当前对话是否到了自然的停止点：

【剧情背景】：{sentence_context}
【对话目标】：{dialogue_goal}
【对话历史】：
{history}

请判断：这段对话是否已经自然结束？考虑因素包括：
- 对话目标是否基本达成
- 是否避免了明显的重复或拖沓
- 角色间的互动是否达到了合理的程度

可选角色：{candidate_characters}

格式：{{"should_continue": true/false, "next_speaker": "角色名或NONE", "reason": "判断理由"}}"""
        }]
        
        try:
            judge_res = generate_response(judge_prompt)
            judge_data = convert_json(judge_res)
            
            if not isinstance(judge_data, dict):
                print(f"⚠️ 判断返回非dict格式: {type(judge_data)}")
                break
                
            should_continue = judge_data.get("should_continue", False)
            next_speaker = judge_data.get("next_speaker", "NONE")
            reason = judge_data.get("reason", "")
            
            print(f"    📊 内部判断: should_continue={should_continue}, next_speaker={next_speaker}")
                
        except Exception as e:
            print(f"⚠️ 对话判断失败: {e}，结束对话")
            break
        
        # 停止条件
        if (not should_continue or
            next_speaker == "NONE" or 
            next_speaker not in candidate_characters):
            print(f"  🛑 内部判断决定结束对话")
            break

        # 生成发言内容
        prompt_reply = [{
            "role": "system",
            "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
对话目标：{dialogue_goal}
对话历史：{history}
继续说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        try:
            response = generate_response(prompt_reply)
            parsed = convert_json(response)
            
            if isinstance(parsed, dict) and "dialogue" in parsed:
                spoken_line = parsed.get("dialogue", "")
                action = parsed.get("action", "")
                dialogue_list.append({
                    "speaker": next_speaker,
                    "dialogue": spoken_line,
                    "action": action
                })
                history += f"{next_speaker}: {spoken_line}\n"
                print(f"  {len(dialogue_list)}. {next_speaker}: {spoken_line[:60]}...")
        except Exception as e:
            print(f"⚠️ 发言生成失败: {e}，跳过这轮")
            continue
    
    if round_count >= SAFETY_LIMIT:
        print(f"⚠️ 达到安全保底({SAFETY_LIMIT}轮)")
    
    final_rounds = len(dialogue_list)
    deviation = final_rounds - expected_rounds
    
    print(f"  ✅ 生成了{final_rounds}条对话，进行了{external_judge_calls}次Claude外部检查")
    print(f"  📊 偏差分析: 预期{expected_rounds}轮 → 实际{final_rounds}轮 (偏差{deviation:+d})")
    
    return dialogue_list, expected_rounds, external_judge_calls


# Cell: 测试多模型验证效果
def test_multi_model_validation():
    """测试多模型验证对之前失控场景的控制效果"""
    
    print("🧪 测试多模型验证版本:")
    print("="*80)
    
    # 测试之前失控的场景
    test_scenarios = [
        {
            "sentence": "耳边,小白的虚拟声音低语,分析前方能量波动与生物活动数据",
            "characters": ['小红帽', '人工智能助手（小白）'],
            "description": "之前21轮失控的场景"
        },
        {
            "sentence": "小红帽目光坚定,脚步沉稳,她机警地避开高风险区域,用便携扫描仪采集周围样本",
            "characters": ['小红帽', '人工智能助手（小白）'],
            "description": "之前14轮的场景"
        },
        {
            "sentence": "穿行之中,她展现出过人的判断力与勇气",
            "characters": ['小红帽', '人工智能助手（小白）'],
            "description": "之前13轮的场景"
        }
    ]
    
    results = []
    
    for i, scenario in enumerate(test_scenarios):
        print(f"\n📋 测试场景 {i+1}: {scenario['description']}")
        print(f"📝 句子: {scenario['sentence'][:50]}...")
        print(f"👥 角色: {scenario['characters']}")
        print("-" * 60)
        
        dialogue_list, expected, external_checks = generate_dialogue_test_with_external_judge(
            scenario['sentence'],
            scenario['characters']
        )
        
        actual_rounds = len(dialogue_list)
        deviation = actual_rounds - expected
        
        results.append({
            "scenario": i+1,
            "expected": expected,
            "actual": actual_rounds,
            "deviation": deviation,
            "external_checks": external_checks,
            "description": scenario['description']
        })
        
        print(f"\n🎯 场景{i+1}结果汇总:")
        print(f"   预期轮数: {expected}")
        print(f"   实际轮数: {actual_rounds}")
        print(f"   偏差: {deviation:+d}")
        print(f"   外部检查: {external_checks}次")
        
        if actual_rounds <= 8:  # 相比之前的21轮有明显改善
            print(f"   ✅ 控制良好 (vs 之前的超长对话)")
        else:
            print(f"   ⚠️  仍可能过长")
    
    # 总体统计
    print(f"\n📊 多模型验证测试总结:")
    print("=" * 80)
    avg_expected = sum(r['expected'] for r in results) / len(results)
    avg_actual = sum(r['actual'] for r in results) / len(results)
    avg_deviation = sum(r['deviation'] for r in results) / len(results)
    avg_checks = sum(r['external_checks'] for r in results) / len(results)
    
    print(f"平均预期轮数: {avg_expected:.1f}")
    print(f"平均实际轮数: {avg_actual:.1f}")
    print(f"平均偏差: {avg_deviation:+.1f}")
    print(f"平均外部检查次数: {avg_checks:.1f}")
    
    if avg_actual <= 6:
        print("✅ 多模型验证成功控制了对话长度")
    else:
        print("⚠️ 仍需进一步优化控制策略")
    
    return results

# 运行测试
test_results = test_multi_model_validation()

🧪 测试多模型验证版本:

📋 测试场景 1: 之前21轮失控的场景
📝 句子: 耳边,小白的虚拟声音低语,分析前方能量波动与生物活动数据...
👥 角色: ['小红帽', '人工智能助手（小白）']
------------------------------------------------------------

🔍 开始生成对话，候选角色: ['小红帽', '人工智能助手（小白）']
📝 测试句子: 耳边,小白的虚拟声音低语,分析前方能量波动与生物活动数据...
原始 content: {"goal": "通过对话让小白向小红帽报告前方的能量波动与生物活动分析结果，并引导小红帽做出下一步行动决策。", "expected_rounds": 2}...
🎯 对话目标: 通过对话让小白向小红帽报告前方的能量波动与生物活动分析结果,并引导小红帽做出下一步行动决策.
📊 预期轮数: 2
原始 content: {"dialogue": "小白，前方的能量波动和生物活动数据分析得怎么样了？能详细说说吗？", "action": "请求小白汇报前方能量波动与生物活动分析结果"}...
  1. 小红帽: 小白,前方的能量波动和生物活动数据分析得怎么样了？能详细说说吗？...
原始 content: {"should_continue": true, "next_speaker": "人工智能助手（小白）", "reason": "对话目标尚未达成。小红帽刚刚提出了对前方能量波动和生物活动数据的分析请求，还没有收到小白的详细分析和建议，因此对话需要继续，由小白回应分析结果并引导小红帽做决策。"}...
    📊 内部判断: should_continue=True, next_speaker=人工智能助手（小白）
原始 content: {"dialogue": "小红帽，根据最新的传感数据，前方50米处检测到异常高频能量波动，同时生物活动信号呈现不规律波动，可能存在未知生物或能量源。你需要决定是否靠近进一步侦查，还是选择绕行以确保安全。", "action": "建议小红帽评估风险后，做出靠近侦查或绕行的决策。"}...
  2. 人工智能助手（小白）: 小红帽,根据最新的传感数据,前方50米处检测到异常高频能量波动,同时生物活动信号呈现不规律波动,可能存在未知生

In [33]:
# 对比你修改前后的效果
def compare_old_vs_new_version():
    """对比修改前后的效果"""
    
    # 选择一个测试句子
    test_sentence = "小红帽检查着医疗芯片的完整性,确保这个救命物品能安全送达祖母手中"
    test_chars = ['小红帽', '智能助手（小智）']
    
    print("🔄 对比测试: 旧版 vs 新版")
    print(f"测试句子: {test_sentence}")
    print(f"测试角色: {test_chars}")
    
    # 如果你还保留着旧版函数，可以这样对比
    # old_result = generate_dialogue_for_insertion_original(test_sentence, test_chars, [], real_characters)
    
    # 新版本
    from src.generation.dialogue_inserter import generate_dialogue_for_insertion
    new_result = generate_dialogue_for_insertion(test_sentence, test_chars, [], real_characters)
    
    print(f"\n📊 结果对比:")
    # print(f"旧版本: {len(old_result)}轮对话")
    print(f"新版本: {len(new_result)}轮对话")
    
    print(f"\n💬 新版本对话内容:")
    for i, d in enumerate(new_result):
        print(f"  {i+1}. {d['speaker']}: {d['dialogue'][:80]}...")
        if d.get('action'):
            print(f"     动作: {d['action'][:60]}...")
    
    return new_result

# 运行对比
comparison_result = compare_old_vs_new_version()

🔄 对比测试: 旧版 vs 新版
测试句子: 小红帽检查着医疗芯片的完整性,确保这个救命物品能安全送达祖母手中
测试角色: ['小红帽', '智能助手（小智）']

🔍 开始生成对话，候选角色: ['小红帽', '智能助手（小智）']
原始 content: {"goal": "通过对话表达小红帽对医疗芯片安全性和重要性的关切，以及她与智能助手协作检查芯片状态的过程，强调芯片的关键作用和任务的紧迫性。", "expected_rounds": 3}...
原始 content: {"dialogue": "小智，我们再仔细检查一次这个医疗芯片的完整性吧，这可是祖母的救命稻草，绝对不能出任何差错。", "action": "小红帽拿出医疗芯片，将其放在读卡设备上，等待智能助手进行扫描与诊断。"}...
原始 content: {"should_continue": true, "next_speaker": "智能助手（小智）", "reason": "当前对话仅有小红帽单方面的发言，提出了要检查医疗芯片的请求，并表达了任务重要性，但智能助手（小智）还没有回应或参与互动。为了实现对话目标（强调芯片关键作用和任务紧迫性，以及双方协作检查过程），需要小智回应并配合检查，互动尚未完成。"}...
原始 content: {"dialogue": "好的，小红帽，我正在进行全面扫描。芯片外壳无损，数据传输正常，所有功能模块均处于最佳状态。你可以放心，但我们要继续保持警惕，途中任何异常都要及时处理。", "action": "启动芯片安全监测系统，实时追踪芯片状态，并设置异常警报。"}...
原始 content: {"should_continue": true, "next_speaker": "小红帽", "reason": "对话目前仅完成了对芯片状态的初步确认，角色间互动有明确目标，但小红帽还未表达检查后的感受或下一步计划，且与智能助手协作的过程尚未充分展开。继续深入可以让任务紧迫性和芯片重要性得到更丰富的表达，对话尚未自然结束。"}...
原始 content: {"dialogue": "谢谢你，小智。每一步都不能马虎，这枚芯片承载着祖母的希望。如果途中出现干扰或芯片异常，请立即通知我，我们要随时准备应对。", "action": "小红帽将芯片仔细收好，并调整随

In [25]:
# 批量测试多种场景组合
def batch_test_scenarios():
    """批量测试不同角色组合和场景"""
    
    test_scenarios = [
        {
            "sentence": "机械大灰狼的红眼在森林深处闪烁，发出威胁性的嗡鸣声",
            "characters": ['小红帽', '机械大灰狼'],
            "type": "冲突场景"
        },
        {
            "sentence": "银河巡逻队队长接收到小红帽的求救信号，立即部署救援计划",  
            "characters": ['银河巡逻队队长', '银河巡逻队队员甲'],
            "type": "救援场景"
        },
        {
            "sentence": "智能助手小智分析着周围环境的数据，为小红帽提供导航建议",
            "characters": ['小红帽', '智能助手（小智）'],
            "type": "技术支持场景"
        }
    ]
    
    results = []
    for i, scenario in enumerate(test_scenarios):
        print(f"\n{'='*60}")
        print(f"🎬 场景 {i+1}: {scenario['type']}")
        print(f"📝 句子: {scenario['sentence']}")
        print(f"👥 角色: {scenario['characters']}")
        print('='*60)
        
        result = generate_dialogue_for_insertion(
            scenario['sentence'],
            scenario['characters'], 
            [],
            real_characters
        )
        
        results.append({
            "scenario": scenario,
            "rounds": len(result),
            "dialogue": result
        })
        
        print(f"📊 结果: {len(result)}轮对话")
    
    return results

# 运行批量测试
batch_results = batch_test_scenarios()


🎬 场景 1: 冲突场景
📝 句子: 机械大灰狼的红眼在森林深处闪烁，发出威胁性的嗡鸣声
👥 角色: ['小红帽', '机械大灰狼']

🔍 开始生成对话，候选角色: ['小红帽', '机械大灰狼']
原始 content: {"goal": "通过对话展现小红帽对机械大灰狼的恐惧与警觉，以及机械大灰狼对小红帽的威胁和意图。", "expected_rounds": 3}...
原始 content: {"dialogue": "这是什么声音？那双红色的眼睛……它在盯着我吗？", "action": "小红帽紧紧攥住手里的篮子，慢慢后退，神情惊恐地望向机械大灰狼的方向。"}...
原始 content: {"should_continue": true, "next_speaker": "机械大灰狼", "reason": "当前只有小红帽的独白，尚未形成与机械大灰狼的互动。根据对话目标，需要展现机械大灰狼对小红帽的威胁和意图。对话还没有达到合理的交流深度，需要机械大灰狼回应以推动剧情发展。"}...
原始 content: {"dialogue": "小女孩，你的心跳声在我雷达里清晰无比。别想逃跑，我的金属爪子比你想象的还要快。", "action": "机械大灰狼缓缓向小红帽逼近，红眼的光芒在黑暗中越来越亮，爪子摩擦着地面发出刺耳的声音。"}...
原始 content: {"should_continue": true, "next_speaker": "小红帽", "reason": "对话目前还未达到自然的停止点。小红帽刚刚表达了恐惧和疑问，机械大灰狼则展现了威胁和意图，但小红帽尚未做出进一步反应（如请求帮助、寻求逃脱、进一步对抗或表达情绪），角色间的互动还未完全展开，也未形成完整的交流。继续一轮小红帽的回应有助于推动剧情发展，使对话更为完整和自然。"}...
原始 content: {"dialogue": "你、你到底想做什么？不要靠近我！", "action": "小红帽往后退了一步，紧紧抓住篮子，眼中满是惊恐。"}...
原始 content: {"should_continue": true, "next_speaker": "机械大灰狼", "reason": "当前对话已经展现了小红帽的恐惧和机械大灰狼的威胁，但机械大灰狼的意图尚未明确表达

In [26]:
# Cell: 对比原版vs新版的对话插入密度
def compare_dialogue_insertion_density():
    """对比原版和新版在相同数据上的对话插入率"""
    
    # 使用相同的测试章节
    test_chapter = real_story[0] if real_story else None
    if not test_chapter:
        print("没有测试数据")
        return
    
    plot = test_chapter.get("plot", "")
    sentences = split_plot_into_sentences(plot)
    
    print(f"📝 测试章节: {test_chapter.get('title', 'Unknown')}")
    print(f"📊 总句子数: {len(sentences)}")
    print("\n" + "="*80)
    
    # 测试原版analyze_dialogue_insertions
    print("🔍 原版analyze_dialogue_insertions分析...")
    original_marks = dialogue_inserter.analyze_dialogue_insertions(sentences, real_characters)
    
    if original_marks:
        original_need_dialogue = sum(1 for m in original_marks if m.get('need_to_action') == 1)
        original_rate = original_need_dialogue / len(original_marks) * 100
        print(f"原版结果: {original_need_dialogue}/{len(original_marks)}句需要对话 ({original_rate:.1f}%)")
    else:
        print("原版分析失败")
        return
    
    # 测试新版（从v2结果中提取相同章节）
    print(f"\n🔍 新版analyze_dialogue_insertions_v2分析...")
    
    # 从之前的sentence_results中找到相同章节的数据
    chapter_id = test_chapter.get("chapter_id", "")
    chapter_sentences = [s for s in sentence_results if s.get('chapter_id') == chapter_id]
    
    if chapter_sentences:
        new_need_dialogue = sum(1 for s in chapter_sentences if s.get('need_to_action') == 1)
        new_rate = new_need_dialogue / len(chapter_sentences) * 100
        print(f"新版结果: {new_need_dialogue}/{len(chapter_sentences)}句需要对话 ({new_rate:.1f}%)")
    else:
        print("新版数据不足，重新生成...")
        # 重新运行单章节测试
        single_chapter_result = dialogue_inserter.analyze_dialogue_insertions_v2([test_chapter], real_characters)
        chapter_sentences = single_chapter_result[1]  # sentence_results
        new_need_dialogue = sum(1 for s in chapter_sentences if s.get('need_to_action') == 1)
        new_rate = new_need_dialogue / len(chapter_sentences) * 100
        print(f"新版结果: {new_need_dialogue}/{len(chapter_sentences)}句需要对话 ({new_rate:.1f}%)")
    
    # 对比分析
    print(f"\n📊 密度对比:")
    print(f"原版对话密度: {original_rate:.1f}%")
    print(f"新版对话密度: {new_rate:.1f}%")
    print(f"差异: {new_rate - original_rate:+.1f}个百分点")
    
    if new_rate < original_rate:
        print("⚠️ 新版对话密度明显降低，可能遗漏了一些对话机会")
    elif new_rate > original_rate:
        print("📈 新版对话密度提高，可能更积极地插入对话")
    else:
        print("➡️ 对话密度基本相同")
    
    return {
        "original_rate": original_rate,
        "new_rate": new_rate,
        "original_marks": original_marks,
        "new_sentences": chapter_sentences
    }

# 运行密度对比
density_comparison = compare_dialogue_insertion_density()

📝 测试章节: 银河救援
📊 总句子数: 5

🔍 原版analyze_dialogue_insertions分析...

 analyze_dialogue_insertions 原始返回内容：
 [
  {
    "sentence": "小红帽走下飞船,抱紧装有医疗纳米芯片的小型包裹,警惕地环顾四周.",
    "need_to_action": 0,
    "actor_list": []
  },
  {
    "sentence": "她的智能助手小智在耳畔低声汇报周遭能量波动异常,令她不禁加快脚步.",
    "need_to_action": 1,
    "actor_list": ["小红帽", "智能助手（小智）"]
  },
  {
    "sentence": "就在此时,银河巡逻队的队长带领队员们现身,身着制式装甲,神情严肃.",
    "need_to_action": 0,
    "actor_list": []
  },
  {
    "sentence": "队长用简短而坚定的口吻要求小红帽说明来意,并警告她该区域刚刚检测到异常信号,可能存在安全威胁.",
    "need_to_action": 1,
    "actor_list": ["银河巡逻队队长", "小红帽"]
  },
  {
    "sentence": "双方初次见面,彼此保持距离,但在银河深处的这场紧急救援中,命运已悄然交织.",
    "need_to_action": 0,
    "actor_list": []
  }
] 

原始 content: [
  {
    "sentence": "小红帽走下飞船,抱紧装有医疗纳米芯片的小型包裹,警惕地环顾四周.",
    "need_to_action": 0,
    "actor_list": []
  },
  {
    "sentence": "她的智能助手小智在耳畔低声汇报周遭能量波动异常,令她不禁加快脚步.",
    "need_to_action": 1,
    "actor_list": ["小红帽", "智能助手（小智）"]
  },
  {
    "sentence": "就在此时,银河巡逻队的队长带领队员们现身,身着制式装甲,神情严肃.",
  

In [27]:
# Cell: 测试是否在"演戏"
def test_without_expected_rounds():
    """移除expected_rounds信息，看实际轮数是否还这么准确"""
    
    # 修改judge_prompt，不提供预期轮数
    def generate_dialogue_blind_test(sentence_context, candidate_characters):
        dialogue_list = []
        history = ""
        
        # 第一个发言人（保持不变）
        speaker = candidate_characters[0]
        prompt_first = [{
            "role": "system",
            "content": f"""你是 {speaker}，基于以下剧情做出第一句发言：
剧情背景：{sentence_context}
其他角色：{[c for c in candidate_characters if c != speaker]}
格式：{{"dialogue": "...", "action": "..."}}"""
        }]
        
        response = generate_response(prompt_first)
        parsed = convert_json(response)
        
        if isinstance(parsed, dict) and "dialogue" in parsed:
            dialogue_list.append({
                "speaker": speaker,
                "dialogue": parsed.get("dialogue", ""),
                "action": parsed.get("action", "")
            })
            history += f"{speaker}: {parsed.get('dialogue', '')}\n"
        
        # 循环判断（完全不提供预期轮数）
        round_count = 0
        while round_count < 15:  # 更高的安全上限
            round_count += 1
            
            # 关键：不提供任何预期信息
            judge_prompt = [{
                "role": "system",
                "content": f"""你是故事编剧。分析当前对话是否到了自然的停止点：

【剧情背景】：{sentence_context}
【对话历史】：{history}

这段对话是否已经自然结束？考虑：
- 对话是否有了相对完整的交流
- 是否避免了明显的重复或拖沓
- 角色间的互动是否达到了合理的程度

格式：{{"should_continue": true/false, "next_speaker": "角色名或NONE", "reason": "判断理由"}}"""
            }]
            
            judge_res = generate_response(judge_prompt)
            judge_data = convert_json(judge_res)
            
            should_continue = judge_data.get("should_continue", False)
            next_speaker = judge_data.get("next_speaker", "NONE")
            
            if (not should_continue or 
                next_speaker == "NONE" or 
                next_speaker not in candidate_characters):
                break
            
            # 生成下一句对话
            prompt_reply = [{
                "role": "system",
                "content": f"""你是 {next_speaker}，基于剧情：{sentence_context}
对话历史：{history}
继续说一句话，格式：{{"dialogue": "...", "action": "..."}}"""
            }]
            
            response = generate_response(prompt_reply)
            parsed = convert_json(response)
            
            if isinstance(parsed, dict) and "dialogue" in parsed:
                dialogue_list.append({
                    "speaker": next_speaker,
                    "dialogue": parsed.get("dialogue", ""),
                    "action": parsed.get("action", "")
                })
                history += f"{next_speaker}: {parsed.get('dialogue', '')}\n"
        
        return dialogue_list
    
    # 测试相同场景
    test_sentence = "小红帽检查着医疗芯片的完整性,确保这个救命物品能安全送达祖母手中"
    test_chars = ['小红帽', '智能助手（小智）']
    
    print("🎭 盲测：不提供expected_rounds")
    blind_result = generate_dialogue_blind_test(test_sentence, test_chars)
    
    print("🔍 新版：提供expected_rounds")
    normal_result = generate_dialogue_for_insertion(test_sentence, test_chars, [], real_characters)
    
    print(f"\n📊 演戏检验结果:")
    print(f"盲测轮数: {len(blind_result)}")
    print(f"正常轮数: {len(normal_result)}")
    print(f"差异: {abs(len(blind_result) - len(normal_result))}轮")
    
    if abs(len(blind_result) - len(normal_result)) <= 1:
        print("✅ 差异很小，LLM判断较为独立")
    else:
        print("⚠️ 差异明显，可能存在'演戏'行为")
    
    return blind_result, normal_result

# 运行演戏检验
blind_test, normal_test = test_without_expected_rounds()

🎭 盲测：不提供expected_rounds
原始 content: {"dialogue": "小智，医疗芯片看起来完好无损，我们必须保证它一路安全送到祖母那里。你帮我再检测一次芯片的数据好吗？", "action": "小红帽把医疗芯片放在检测仪上，等待小智进行扫描。"}...
原始 content: {"should_continue": true, "next_speaker": "小智", "reason": "当前对话并未自然结束。小红帽刚刚请求小智帮助她再次检测芯片的数据，这是一个明确的行动请求。合理的互动流程是小智回应或执行这一请求，因此需要小智的下一步回应，才能形成完整的交流。当前并未出现明显的重复或拖沓，也没有达到合理的对话终点。"}...
🔍 新版：提供expected_rounds

🔍 开始生成对话，候选角色: ['小红帽', '智能助手（小智）']
原始 content: {"goal": "通过对话体现小红帽对医疗芯片安全性的关切，以及与智能助手确认芯片状态和运输安全措施，突出任务的重要性和紧迫感。", "expected_rounds": 3}...
原始 content: {"dialogue": "小智，帮我再扫描一次医疗芯片的功能模块，我必须确认它没有任何损坏或异常。祖母的病情不能再有任何意外，芯片的安全至关重要。", "action": "小红帽拿出医疗芯片，对准扫描仪，请求小智进行详细检测"}...
原始 content: {"should_continue": true, "next_speaker": "智能助手（小智）", "reason": "当前只有小红帽单方面发言，尚未收到智能助手对芯片状态的确认与反馈，未形成完整的互动。根据对话目标，需要体现小红帽对芯片安全的关切，并与智能助手确认芯片状态，因此还需继续至少一轮由智能助手回应，进一步推动剧情发展。"}...
原始 content: {"dialogue": "正在进行深度扫描，请稍等……扫描完成，所有功能模块工作正常，无损坏迹象。小红帽，你是否需要激活芯片的应急保护协议，以应对途中可能的突发状况？", "action": "执行医疗芯片深度扫描，报告结果，并建议激活应急保护协议。"}...
原始 content: {"should_continue": tru

In [28]:
# Cell: 故意给错误的预期，看是否被误导
def test_with_wrong_expectations():
    """故意给错误的expected_rounds，看LLM是否被误导"""
    
    # 修改goal_prompt，故意返回错误预期
    def generate_dialogue_with_fake_expectation(sentence_context, candidate_characters, fake_rounds):
        # 直接设置假的预期轮数，不经过LLM预测
        dialogue_goal = "推进剧情"
        expected_rounds = fake_rounds
        
        print(f"🎯 故意设定预期: {expected_rounds}轮")
        
        # 剩余逻辑和你的新版本相同...
        dialogue_list = []
        history = ""
        
        speaker = candidate_characters[0]
        # ...（省略第一句生成代码，和原版相同）
        
        # 关键：在judge_prompt中使用假的预期
        round_count = 0
        while round_count < 15:
            round_count += 1
            
            judge_prompt = [{
                "role": "system", 
                "content": f"""【预期轮数】：{expected_rounds}
【当前轮数】：{round_count}
... # 其余和你的代码相同
"""
            }]
            # ... 继续执行判断逻辑
            
        return dialogue_list
    
    test_sentence = "机械大灰狼突然出现，威胁小红帽"
    test_chars = ['小红帽', '机械大灰狼']
    
    print("🧪 测试不同的假预期对实际轮数的影响:")
    
    # 测试1：预期1轮
    result_1 = generate_dialogue_with_fake_expectation(test_sentence, test_chars, 1)
    print(f"预期1轮 → 实际{len(result_1)}轮")
    
    # 测试2：预期8轮  
    result_8 = generate_dialogue_with_fake_expectation(test_sentence, test_chars, 8)
    print(f"预期8轮 → 实际{len(result_8)}轮")
    
    # 对比正常预测
    normal = generate_dialogue_for_insertion(test_sentence, test_chars, [], real_characters)
    print(f"正常预测 → 实际{len(normal)}轮")
    
    # 分析是否被误导
    if len(result_1) < len(result_8):
        print("⚠️ LLM被预期轮数误导了，可能在'演戏'")
    else:
        print("✅ LLM判断相对独立，没有明显的预期偏向")

# test_with_wrong_expectations()

In [29]:
# Cell: 分析LLM的停止理由
def analyze_stop_reasons():
    """分析LLM在不同轮数时的停止理由"""
    
    # 从统计文件中读取最近的判断理由
    import json
    stats_file = "/Users/haha/Story/data/output/logs/dialogue_generation.log"
    
    if os.path.exists(stats_file):
        with open(stats_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        # 提取DIALOGUE_END的理由
        stop_reasons = []
        for line in lines:
            if "LLM_JUDGE" in line and "reason=" in line:
                # 简单提取理由部分
                reason_start = line.find("reason=") + 7
                reason_end = line.find("...", reason_start)
                if reason_end > reason_start:
                    reason = line[reason_start:reason_end]
                    stop_reasons.append(reason)
        
        print("📋 最近的LLM判断理由:")
        for i, reason in enumerate(stop_reasons[-5:]):  # 最后5条
            print(f"{i+1}. {reason}")
        
        # 检查是否有轮数相关的理由
        rounds_influenced = sum(1 for r in stop_reasons if "轮" in r or "预期" in r)
        print(f"\n📊 受轮数影响的判断: {rounds_influenced}/{len(stop_reasons)} ({rounds_influenced/len(stop_reasons)*100:.1f}%)")
    else:
        print("日志文件不存在")

analyze_stop_reasons()

📋 最近的LLM判断理由:


ZeroDivisionError: division by zero